In [67]:
import polars as pl
import os
from tqdm import tqdm
import re
from dataclasses import dataclass
from typing import List, Optional
from IPython.display import Markdown
import kagglehub
import sys
sys.path.append('..')
from synthetic_data.screenplay_parser import ScreenplayParser

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
scripts_corpus_path = kagglehub.dataset_download("veeralakrishna/imsdb-movie-scripts")
print(scripts_corpus_path)
scripts_pqt_path = os.path.join(scripts_corpus_path, "movie_scripts.parquet")
scripts_pqt = pl.read_parquet(scripts_pqt_path)

In [ ]:
for i in range(10):
    display(Markdown(f"## {scripts_pqt[i]['Movie'][0]}"))
    print(scripts_pqt[i]['Script'][0][0:1000])

In [77]:


i, n_failed = 0, 0
for row in tqdm(scripts_pqt.iter_rows(named=True)):
    parser = ScreenplayParser(row['Script'])
    scenes = parser.parse()
    n_chars, n_chars_with_lines = len(scenes), len(parser.character_line_counts)
    # print(f"Script {i} has {n_chars} scenes and {n_chars_with_lines} characters with lines")
    if n_chars_with_lines == 0 and n_chars == 0:
        # print(row['Script'])
        n_failed += 1
    # for scene in scenes:
    #     display(Markdown(f"### {scene.heading.content}"))
    #     for elem in scene.elements:
    #         if elem.type == 'dialogue':
    #             if not elem.dialogue_data:
    #                 raise ValueError("Dialogue element has no dialogue data")
    #             display(Markdown(f"**{elem.dialogue_data.character}**: {elem.dialogue_data.content}"))
    i += 1

print(f"Failed to parse {n_failed} scripts")

1282it [00:04, 293.05it/s]

Failed to parse 380 scripts
